In [ ]:
from __future__ import division
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import lightgbm as lgb
import time
import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
action_tr = pd.read_csv('../data/input/train/action_train.csv')  # 用户行为数据
order_future_tr = pd.read_csv('../data/input/train/orderFuture_train.csv')  # 待预测数据
order_history_tr = pd.read_csv('../data/input/train/orderHistory_train.csv')  # 用户历史订单数据
user_comment_tr = pd.read_csv('../data/input/train/userComment_train.csv')  # 用户评论数据
user_profile_tr = pd.read_csv('../data/input/train/userProfile_train.csv')  # 用户个人信息

action_te = pd.read_csv('../data/input/test/action_test.csv')
order_future_te = pd.read_csv('../data/input/test/orderFuture_test.csv')
order_history_te = pd.read_csv('../data/input/test/orderHistory_test.csv')
user_comment_te = pd.read_csv('../data/input/test/userComment_test.csv')
user_profile_te = pd.read_csv('../data/input/test/userProfile_test.csv')

In [ ]:
action = pd.concat([action_tr, action_te], axis=0).reset_index(drop=True)
order_history = pd.concat([order_history_tr, order_history_te], axis=0).reset_index(drop=True)
user_comment = pd.concat([user_comment_tr, user_comment_te], axis=0).reset_index(drop=True)
user_profile = pd.concat([user_profile_tr, user_profile_te], axis=0).reset_index(drop=True)

In [ ]:
def get_user_profile_feature(df):
    le = preprocessing.LabelBinarizer()
    encoder = le.fit_transform(df.gender.fillna('_NA_').values)
    encoder_col = ['gender_%d'%i for i in range(encoder.shape[1])]
    df1 = pd.DataFrame(encoder, columns = encoder_col)
    df1['userid'] = df['userid'].values
    f = df1
    
    le = preprocessing.LabelBinarizer()
    encoder = le.fit_transform(df.province.fillna('_NA_').values)
    encoder_col = ['province_%d'%i for i in range(encoder.shape[1])]
    df1 = pd.DataFrame(encoder, columns = encoder_col)
    df1['userid'] = df['userid'].values
    f = pd.merge(f,df1, on = 'userid', how = 'left')
    
    le = preprocessing.LabelBinarizer()
    encoder = le.fit_transform(df.age.fillna('_NA_').values)
    encoder_col = ['age_%d'%i for i in range(encoder.shape[1])]
    df1 = pd.DataFrame(encoder, columns = encoder_col)
    df1['userid'] = df['userid'].values
    f = pd.merge(f,df1, on = 'userid', how = 'left')
    return f

In [ ]:
def get_action_feature(data):
    df = copy.deepcopy(data)
    #---all action feature
    result = df.groupby(['userid','actionType'])['actionTime'].count().reset_index().rename(
        columns = {'actionTime':'actionNum'}).pivot('userid','actionType','actionNum').apply(lambda x: x/np.sum(x))
    result.columns = ['action_type_num%d'%i for i in range(result.shape[1])]
    result = result.reset_index()
    
    #---get feature by unix time
    for window in [6]:
        print window
        print 'actiontype'
        f = df.groupby('userid').apply(lambda x: x.sort_values('actionTime', ascending = False).head(window)).reset_index(drop = True).rename(
            columns={'actionType': 'actionType_last', 'actionTime': 'actionTime_last'})
        f2 = pd.concat([f,f.groupby('userid').rank(method = 'first').astype('int').reset_index().rename(
                columns={'actionTime_last': 'actionTime_last_rank'})['actionTime_last_rank']],axis = 1)
        
        ff1 = f2.pivot('userid','actionTime_last_rank','actionType_last')
        ff1_diff = ff1.diff(1, axis = 1)
        ff1_diff.columns = ['last%d_type_1diff%d'%(window,i) for i in range(ff1_diff.shape[1])]
        ff1_diff = ff1_diff.iloc[:,1:].reset_index()
        result = pd.merge(result, ff1_diff, on = 'userid', how = 'left')
        
        ff1['last%d_type_max'%window] = ff1.max(axis = 0)
        ff1['last%d_type_min'%window] = ff1.min(axis = 0)
        ff1['last%d_type_meam'%window] = ff1.mean(axis = 0)
        ff1['last%d_type_median'%window] = ff1.median(axis = 0)
        ff1['last%d_type_std'%window] = ff1.std(axis = 0)
        ff1['last%d_type_sum'%window] = ff1.sum(axis = 0)
        
        ff1.columns = ['last%d_type%d'%(window,i) for i in range(ff1.shape[1])]
        ff1 = ff1.reset_index()
        
        result = pd.merge(result, ff1, on = 'userid', how = 'left')
        

        print 'actiontime'
        ff2 = f2.pivot('userid','actionTime_last_rank','actionTime_last')
        ff2_diff = ff2.diff(1, axis = 1)
        ff2_diff.columns = ['last%d_time_1diff%d'%(window,i) for i in range(ff2_diff.shape[1])]
        ff2_diff = ff2_diff.iloc[:,1:].reset_index()
        result = pd.merge(result, ff2_diff, on = 'userid', how = 'left')
        
        

        ff2['last%d_time_max'%window] = ff2.max(axis = 0)
        ff2['last%d_time_min'%window] = ff2.min(axis = 0)
        ff2['last%d_time_meam'%window] = ff2.mean(axis = 0)
        ff2['last%d_time_median'%window] = ff2.median(axis = 0)
        ff2['last%d_time_std'%window] = ff2.std(axis = 0)
        ff2['last%d_time_sum'%window] = ff2.sum(axis = 0)
        
        ff2.columns = ['last%d_time%d'%(window,i) for i in range(ff2.shape[1])]
        ff2 = ff2.reset_index()
        result = pd.merge(result, ff2, on = 'userid', how = 'left')
       
        
        ff = f.groupby(['userid','actionType_last'])['actionTime_last'].count().reset_index().rename(
            columns = {'actionTime_last':'actionNum_last'}).pivot('userid','actionType_last','actionNum_last').apply(lambda x: x/np.sum(x))
        ff.columns = ['last%d_action_type_num%d'%(window,i) for i in range(ff.shape[1])]
        ff = ff.reset_index()
        result = pd.merge(result, ff, on = 'userid', how = 'left')
        
    #---sort every type last 1 action feature
    f = df.groupby(['userid','actionType']).apply(lambda x: x.sort_values('actionTime', ascending = False).head(1)).reset_index(drop = True).rename(
        columns={'actionTime': 'type_actionTime_last'})
    ff3 = f.pivot('userid','actionType','type_actionTime_last')
    
    ff3_diff = ff3.diff(1, axis = 1)
    ff3_diff.columns = ['type_%d_lsttime_diff'%i for i in range(ff3_diff.shape[1])]
    ff3_diff = ff3_diff.iloc[:,1:].reset_index()
    ff3.columns = ['type_%d_lasttime'%i for i in range(ff3.shape[1])]
    ff3 = ff3.reset_index()
    result = pd.merge(result, ff3, on = 'userid', how = 'left')
    
    for t in [1,2,3,4,5,6,7,8,9]:
        print t
        window = 5
        df_type = df[df.actionType == t]
        f = df_type.groupby('userid').apply(lambda x: x.sort_values('actionTime', ascending = False).head(window)).reset_index(drop = True).rename(
            columns={'actionTime': 'actionTime_last'})
        f2 = pd.concat([f,f.groupby('userid').rank(method = 'first').astype('int').reset_index().rename(
                columns={'actionTime_last': 'actionTime_last_rank'})['actionTime_last_rank']],axis = 1)
        
        ff1 = f2.pivot('userid','actionTime_last_rank','actionTime_last')
        ff1_diff = ff1.diff(1, axis = 1)
        ff1_diff.columns = ['last%d_type%d_time_diff%d'%(window,t,i) for i in range(ff1_diff.shape[1])]
        ff1_diff = ff1_diff.iloc[:,1:].reset_index()
        
        ff1['last%d_type%d_max'%(window,t)] = ff1.max(axis = 0)
        ff1['last%d_type%d_min'%(window,t)] = ff1.min(axis = 0)
        ff1['last%d_type%d_meam'%(window,t)] = ff1.mean(axis = 0)
        ff1['last%d_type%d_median'%(window,t)] = ff1.median(axis = 0)
        ff1['last%d_type%d_std'%(window,t)] = ff1.std(axis = 0)
        ff1['last%d_type%d_sum'%(window,t)] = ff1.sum(axis = 0)
        
        ff1.columns = ['last%d_type%d_time%d'%(window,t,i) for i in range(ff1.shape[1])]
        ff1 = ff1.reset_index()
        
        result = pd.merge(result, ff1, on = 'userid', how = 'left')
        result = pd.merge(result, ff1_diff, on = 'userid', how = 'left')
    #--get feature by date
#     print 'date feature'
#     df['date'] = pd.to_datetime(df['actionTime'],unit='s').dt.date
#     for window in [40]:
#         print window
#         df_select = df[df.date >= pd.bdate_range(end=df.date.max(), periods=window).date[0]]
#         print df_select.shape
#         f = df_select.groupby('userid').apply(lambda x: x.sort_values('actionTime', ascending = False).head(6)).reset_index(drop = True).rename(
#                 columns={'actionType': 'actionType_last', 'actionTime': 'actionTime_last'})
#         f2 = pd.concat([f,f.groupby('userid').rank(method = 'first').astype('int').reset_index().rename(
#                 columns={'actionTime_last': 'actionTime_last_rank'})['actionTime_last_rank']],axis = 1)
        
#         ff1 = f2.pivot('userid','actionTime_last_rank','actionType_last')
#         ff1_diff = ff1.diff(1, axis = 1)
#         ff1_diff.columns = ['lastdate%d_type_diff%d'%(window,i) for i in range(ff1_diff.shape[1])]
#         ff1_diff = ff1_diff.iloc[:,1:].reset_index()
        
#         ff1['lastdate%d_type_max'%window] = ff1.max(axis = 0)
#         ff1['lastdate%d_type_min'%window] = ff1.min(axis = 0)
#         ff1['lastdate%d_type_meam'%window] = ff1.mean(axis = 0)
#         ff1.columns = ['lastdate%d_type%d'%(window,i) for i in range(ff1.shape[1])]
#         ff1 = ff1.reset_index()
        
#         result = pd.merge(result, ff1, on = 'userid', how = 'left')
#         result = pd.merge(result, ff1_diff, on = 'userid', how = 'left')
        
#         ff2 = f2.pivot('userid','actionTime_last_rank','actionTime_last')
#         ff2_diff = ff2.diff(1, axis = 1)
#         ff2_diff.columns = ['last%d_time_diff%d'%(window,i) for i in range(ff2_diff.shape[1])]
#         ff2_diff = ff2_diff.iloc[:,1:].reset_index()

#         ff2['last%d_time_max'%window] = ff2.max(axis = 0)
#         ff2['last%d_time_min'%window] = ff2.min(axis = 0)
#         ff2['last%d_time_meam'%window] = ff2.mean(axis = 0)
#         ff2.columns = ['last%d_time%d'%(window,i) for i in range(ff2.shape[1])]
#         ff2 = ff2.reset_index()
#         result = pd.merge(result, ff2, on = 'userid', how = 'left')
#         result = pd.merge(result, ff2_diff, on = 'userid', how = 'left')
        
        
#         ff = f.groupby(['userid','actionType_last'])['actionTime_last'].count().reset_index().rename(
#             columns = {'actionTime_last':'actionNum_last'}).pivot('userid','actionType_last','actionNum_last').apply(lambda x: x/np.sum(x))
#         ff.columns = ['last%d_action_type_num%d'%(window,i) for i in range(ff.shape[1])]
#         ff = ff.reset_index()
#         result = pd.merge(result, ff, on = 'userid', how = 'left')

    
    
    return result

In [ ]:
def get_order_history_feature(data):
    le = preprocessing.LabelBinarizer()
    encoder1 = le.fit(pd.concat([orderHistory_train,orderHistory_test]).country.values)

    le = preprocessing.LabelBinarizer()
    encoder2 = le.fit(pd.concat([orderHistory_train,orderHistory_test]).continent.values)
    
    df = copy.deepcopy(data)
    #--get order feature
    feature = df.groupby('userid')['orderType'].agg(['sum','count']).reset_index().rename(columns={'sum': 'order_num_1', 'count': 'order_num'})
    feature['order_num_0'] = feature['order_num'] - feature['order_num_1']
    feature['order_ratio_0'] = feature['order_num_0'].astype('float')/feature['order_num']
    feature['order_ratio_1'] = feature['order_num_1'].astype('float')/feature['order_num']
    f = copy.deepcopy(feature)
    
    #--get total feature
    feature = df.groupby('userid')['city','country','continent'].count().reset_index().rename(
        columns={'city': 'city_num', 'country': 'country_num','continent':'continent_num'})
    f = pd.merge(f,feature, on = 'userid', how = 'left')
    feature = df[df.orderType == 1].groupby('userid')['city','country','continent'].count().reset_index().rename(
        columns={'city': 'city_num_1', 'country': 'country_num_1','continent':'continent_num_1'})
    f = pd.merge(f,feature, on = 'userid', how = 'left').fillna(0)
    for val in ['city_num', 'country_num', 'continent_num']:
        f[val.split('_')[0]+'_ratio_1'] = f[val+'_1'].astype('float')/f[val]
    #--get country feature
#     le = preprocessing.LabelBinarizer()
    country_encoder = encoder1.transform(df.country.values)
    country_encoder_col = ['country_%d'%i for i in range(country_encoder.shape[1])]
    df1 = pd.DataFrame(country_encoder, columns = country_encoder_col)
    df1['userid'] = df['userid'].values
    feature = df1.groupby('userid')[country_encoder_col].agg(['sum','count']).reset_index()
    f = pd.merge(f,feature, on = 'userid', how = 'left')
    
    #--get continent feature
#     le = preprocessing.LabelBinarizer()
    continent_encoder = encoder2.transform(df.continent.values)
    continent_encoder_col = ['continent_%d'%i for i in range(continent_encoder.shape[1])]
    df1 = pd.DataFrame(continent_encoder, columns = continent_encoder_col)
    df1['userid'] = df['userid'].values
    feature = df1.groupby('userid')[continent_encoder_col].agg(['sum','count']).reset_index()
    f = pd.merge(f,feature, on = 'userid', how = 'left')
    
    
    #--get orderTime last feature
#     df1 = df.groupby('userid').apply(lambda x: x.sort_values('orderTime', ascending = False).head(1)).reset_index(drop = True)[['userid','orderid','orderTime','orderType']]
#     df1.columns = [['userid','last_orderid','last_orderTime','last_orderType']]
#     f = pd.merge(f, df1, on = 'userid',how = 'left')
    
    #--get orderTime last 5 feature
#     df1 = df.groupby('userid').apply(lambda x: x.sort_values('orderTime', ascending = False).head(5)).reset_index(drop = True)[['userid','orderid','orderTime','orderType']]
#     df1.columns = [['userid','last_orderid','last_orderTime','last_orderType']]
#     temp = pd.concat([df1,df1.groupby('userid').rank(method = 'first').astype('int').reset_index().rename(
#             columns={'last_orderTime': 'last_orderTime_rank'})['last_orderTime_rank']],axis = 1)
    
#     ff1 = temp.pivot('userid','last_orderTime_rank','last_orderType')
#     ff1.columns = ['his_type%d'%i for i in range(ff1.shape[1])]
#     ff1 = ff1.reset_index()
#     f = pd.merge(f, ff1, on = 'userid',how = 'left')
    
#     ff2 = temp.pivot('userid','last_orderTime_rank','last_orderTime')
#     ff2.columns = ['his_time%d'%i for i in range(ff2.shape[1])]
#     ff2 = ff2.reset_index()
#     f = pd.merge(f, ff2, on = 'userid',how = 'left')
    
    return f

In [ ]:
def get_user_comment_feature(data):
    df = copy.deepcopy(data)
    feature = df.groupby('userid')['rating'].agg(['max','min','mean','sum','count','median','std']).reset_index().rename(
        columns={'max': 'rate_max', 'min': 'rate_min','mean':'rate_mean','sum':'rate_sum','count':'rate_count','median':'rate_median','std':'rate_std'})

#     mlb = MultiLabelBinarizer()
#     tagsV = []
#     for line in df.tags.values:
#         if line == line:
# #             print line
#             tagsV.append(set(line.split('|')))
#         else:
#             tagsV.append(set(''))
            
    
#     tagsF = mlb.fit_transform(tagsV)
#     name = []
#     for i in range(tagsF.shape[1]):
#         df['tag_%d'%i] = tagsF[:,i]
#         name.append('tag_%d'%i)
    
#     f = df.groupby('userid')[name].agg(['mean','sum','count','std']).reset_index()
    
#     feature = pd.merge(feature, f, on= 'userid', how = 'left') 
    
    
    return feature

In [ ]:
user_profile_feat = get_user_profile_feature(user_profile)
action_feat = get_action_feature(action)
order_history_feat = get_order_history_feature(order_history)
user_comment_feat = get_user_comment_feature(user_comment)

dataset = pd.merge(user_profile_feat, action_feat, on='userid', how='left')
dataset = pd.merge(dataset, order_history_feat, on='userid', how='left')
dataset = pd.merge(dataset, user_comment_feat, on='userid', how='left')

trainset = pd.merge(order_future_tr, dataset, on='userid', how='left')
testset = pd.merge(order_future_te, dataset, on='userid', how='left')

train_feature = trainset.drop(['orderType'], axis=1)
train_label = trainset.orderType.values
test_feature = testset
test_index = testset.userid.values

print train_feature.shape, train_label.shape
print test_feature.shape

In [ ]:
config = {
    'rounds': 10000,
    'folds': 5
}

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'min_sum_hessian_in_leaf': 0.1,
    'learning_rate': 0.01,
    'verbosity': 2,
    'tree_learner': 'feature',
    'num_leaves': 128,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'num_threads': 16,
    'seed': 20
}

In [ ]:
def lgb_cv(train_feature, train_label, params, folds, rounds):
    start = time.clock()
    print train_feature.columns
    dtrain = lgb.Dataset(train_feature, label=train_label)
    num_round = rounds
    print 'run cv: ' + 'round: ' + str(rounds)
    res = lgb.cv(params, dtrain, num_round, nfold=folds, verbose_eval=20, early_stopping_rounds=100)
    elapsed = (time.clock() - start)
    print 'Time used:', elapsed, 's'
    return len(res['auc-mean']), res['auc-mean'][len(res['auc-mean']) - 1]

def lgb_predict(train_feature, train_label, test_feature, rounds, params):
    dtrain = lgb.Dataset(train_feature, label=train_label)
    valid_sets = [dtrain]
    num_round = rounds
    model = lgb.train(params, dtrain, num_round, valid_sets, verbose_eval=50)
    predict = model.predict(test_feature)
    return model, predict

def store_result(test_index, pred, name):
    result = pd.DataFrame({'userid': test_index, 'orderType': pred})
    result.to_csv('../data/output/sub/' + name + '.csv', index=0, columns=['userid', 'orderType'])
    return result

In [100]:
iterations, best_score = lgb_cv(train_feature, train_label, params, config['folds'], config['rounds'])

Index([                 u'userid',                u'gender_0',
                      u'gender_1',                u'gender_2',
                    u'province_0',              u'province_1',
                    u'province_2',              u'province_3',
                    u'province_4',              u'province_5',
       ...
       (u'continent_4', u'count'),   (u'continent_5', u'sum'),
       (u'continent_5', u'count'),                u'rate_max',
                      u'rate_min',               u'rate_mean',
                      u'rate_sum',              u'rate_count',
                   u'rate_median',                u'rate_std'],
      dtype='object', length=373)
run cv: round: 10000
[20]	cv_agg's auc: 0.935916 + 0.00347213
[40]	cv_agg's auc: 0.939652 + 0.00393814
[60]	cv_agg's auc: 0.940786 + 0.00389518
[80]	cv_agg's auc: 0.942099 + 0.00350259
[100]	cv_agg's auc: 0.943578 + 0.00378149
[120]	cv_agg's auc: 0.945306 + 0.00356631
[140]	cv_agg's auc: 0.946317 + 0.0035726
[160]	cv_agg's

In [101]:
model, pred = lgb_predict(train_feature, train_label, test_feature, iterations, params)

[50]	training's auc: 0.961372
[100]	training's auc: 0.966117
[150]	training's auc: 0.970849
[200]	training's auc: 0.974998
[250]	training's auc: 0.979141
[300]	training's auc: 0.982696
[350]	training's auc: 0.985932
[400]	training's auc: 0.988872
[450]	training's auc: 0.991118
[500]	training's auc: 0.993063
[550]	training's auc: 0.994594
[600]	training's auc: 0.995803
[650]	training's auc: 0.996739
[700]	training's auc: 0.997466
[750]	training's auc: 0.998034
[800]	training's auc: 0.998452
[850]	training's auc: 0.998802
[900]	training's auc: 0.999038
[950]	training's auc: 0.999239
[1000]	training's auc: 0.999403
[1050]	training's auc: 0.999536
[1100]	training's auc: 0.999624
[1150]	training's auc: 0.999705
[1200]	training's auc: 0.999772
[1250]	training's auc: 0.999825
[1300]	training's auc: 0.999866
[1350]	training's auc: 0.999895


In [103]:
res = store_result(test_index, pred, '20180121-lgb-%f(r%d)' % (best_score, iterations))